In [ ]:
#!pip install sharepy
import sharepy 
import os
#System variables
MAIL_NUB = os.environ.get('MAIL_NUB')
PASS_MAIL_NUB = os.environ.get('PASS_MAIL_NUB')
s = sharepy.connect("nubsieteocho.sharepoint.com", MAIL_NUB, PASS_MAIL_NUB)

In [ ]:
import smtplib
import pandas as pd
from datetime import datetime, timedelta
from pandas import Timestamp
import excel_formating

from email import encoders
from os.path import basename
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from string import printable

In [ ]:
#import data with the users mails
p = s.getfile("https://nubsieteocho.sharepoint.com/sites/ODAOPS/Shared%20Documents/General/CAB%20Trackers/Coverage%20Advisor%20Tracker%20OFICIAL.xlsx"\
    ,filename = 'Coverage Advisor Tracker OFICIAL.xlsx')
ca = pd.read_excel ('Coverage Advisor Tracker OFICIAL.xlsx', sheet_name='COE Reports')
ca.dropna(axis=0, subset=['Account Name'], inplace=True)
ca.drop(list(ca[ca['Account Name']==0].index), axis=0, inplace=True)
ca.dropna(axis=0, subset=['Request ID'], inplace=True)
ca.drop_duplicates(subset=['Request ID'], inplace=True)

In [ ]:
m = s.getfile("https://nubsieteocho.sharepoint.com/sites/ODAOPS/Shared%20Documents/General/coverage_mails.xlsx"\
    ,filename = 'coverage_mails.xlsx')
mail = pd.read_excel('coverage_mails.xlsx')

In [ ]:
#SMTP Mail variables
today = datetime.strftime(datetime.today(), '%m-%d-%Y')
myMail = 'testNub78@outlook.com'
password = 'EasyP4ssword78'
subjectEmail = 'Coverage error report'

msg1 = """<h3>Hola, %s</h3>
        <p><span style="color: rgb(71, 85, 119);">Parece que tienes varios errores en la informaci&oacute;n registrada en tus cuentas.</span></p>
        <p><span style="color: rgb(71, 85, 119);">Asegurate de revisar el archivo adjunto para validar los campos que est&aacute;n generando conflictos.</span></p>
        <p><span style="color: rgb(71, 85, 119);">Feliz D&iacute;a.</span></p>"""

In [ ]:
def format_date_column(serie):
    format_date = []
    for id, value in serie.iteritems():
        if pd.notna(value) == True:
            try:
                value = datetime.strptime(str(value), '%Y-%m-%d %H:%M:%S')
                format_date.append(value)
            except:
                try:
                    value = datetime.strptime(str(value), '%Y-%m-%d')
                    format_date.append(value)
                except:
                    value = 'Format Error'
                    format_date.append(value)
        else:
            value = 'Empty date'
            format_date.append(value)

    return format_date

In [ ]:
# Request Type

request_type = ca[['Request ID','Request Type']].copy()
request_type = request_type[pd.isna(request_type['Request Type'])]
request_type['Request Type'] = 'Request Type Empty'

In [ ]:
# Join Call

join_call = ca[['Request ID', 'Join Call']].copy()
join_call = join_call[pd.isna(join_call['Join Call'])]
join_call['Join Call'] = 'Join Call Empty'

In [ ]:
# Upcoming Renewal
upcoming_renewal = ca[['Request ID', 'Upcoming Renewal']].copy()

for index, row in upcoming_renewal.iterrows():
    if pd.isna(row['Upcoming Renewal']) == True:
        upcoming_renewal.loc[[index],['Upcoming Renewal']] = 'Empty Value'
    elif len(row['Upcoming Renewal']) > 6:
        upcoming_renewal.loc[[index],['Upcoming Renewal']] = 'Value with space'
    else:    
        upcoming_renewal.loc[[index],['Upcoming Renewal']] = None

upcoming_renewal.dropna(axis=0, subset=['Upcoming Renewal'], inplace=True)

In [ ]:
# Date Received
def column_comparison_2(a,b):
    ans = 'NA'
    if type(a) == datetime:
        if a < Timestamp('2021-10-15 00:00:00'):
            ans = '15/oct error'
        elif type(a) == datetime and type(b) == datetime:
            if a > b:
                ans = 'PAR requested is lower'
            elif a <= b:
                ans = 'correct'
    elif a == 'Empty date':
        ans = 'Empty date'
    elif a == 'Format Error':
        ans = 'Format Error'  
    return ans

date_received = ca[['Request ID', 'Date received', 'PAR Requested']].copy()
date_received['Date received'] = format_date_column(date_received['Date received'])
date_received['PAR Requested'] = format_date_column(date_received['PAR Requested'])
date_received['Date received'] = date_received[['Date received','PAR Requested']].apply(lambda row: column_comparison_2(row[0],row[1]), axis=1)
date_received = date_received[~(date_received['Date received'] == 'correct')]
date_received = date_received[~(date_received['Date received'] == 'NA')]
date_received.drop(axis=1, columns=['PAR Requested'], inplace=True)

In [ ]:
# Coverage Advisor
def coverage_adv(a):
    ans = a
    if pd.isna(a) == True:
        ans = 'Empty Name'
    else: 
        if set(a).difference(printable):
            ans = 'Special Character in Name'
    return ans

cov_adv = ca[['Request ID', 'Coverage Advisor']].copy()
cov_adv['Coverage Advisor'] = cov_adv['Coverage Advisor'].apply(coverage_adv)
cov_adv = cov_adv[(cov_adv['Coverage Advisor'] == 'Empty Name') | (cov_adv['Coverage Advisor'] == 'Special Character in Name')]

In [ ]:
# PAR Requested
def column_comparison_3(a,b):
    ans = b
    if type(a) == datetime and type(b) == datetime:
        if a > b:
            ans = 'date older that Date received'
    return ans

PAR_req = ca[['Request ID', 'Date received', 'PAR Requested']].copy()
PAR_req['Date received'] = format_date_column(PAR_req['Date received'])
PAR_req['PAR Requested'] = format_date_column(PAR_req['PAR Requested'])
PAR_req = PAR_req[~(PAR_req['Date received'] == 'Empty date')]
PAR_req = PAR_req[~(PAR_req['Date received'] == 'Format Error')]
PAR_req['PAR Requested'] = PAR_req[['Date received','PAR Requested']].apply(lambda row: column_comparison_3(row[0],row[1]), axis=1)
PAR_req = PAR_req[PAR_req['PAR Requested'].apply(type) == str]
PAR_req.drop(axis=1, columns=['Date received'], inplace=True)

In [ ]:
# Date 1st contact
d_f_contact = ca[['Request ID', 'Date received', 'Date First Contact']].copy()
d_f_contact['Date received'] = format_date_column(d_f_contact['Date received'])
d_f_contact['Date First Contact'] = format_date_column(d_f_contact['Date First Contact'])
d_f_contact = d_f_contact[~((d_f_contact['Date received'] == 'Empty date') & (d_f_contact['Date First Contact'] == 'Empty date'))]
d_f_contact = d_f_contact[~((d_f_contact['Date received'] == 'Empty date') | (d_f_contact['Date First Contact'] == 'Empty date'))]
format_errors = d_f_contact[d_f_contact['Date First Contact'].apply(type) == str]
d_f_contact = d_f_contact[~(d_f_contact['Date First Contact'].apply(type) == str)]
d_f_contact = d_f_contact[d_f_contact['Date received'] > d_f_contact['Date First Contact']]
d_f_contact['Date First Contact'] = 'date older that Date received'
d_f_contact = pd.concat([d_f_contact, format_errors])
d_f_contact.drop(axis=1, columns=['Date received'], inplace=True)

In [ ]:
# PAR Received

PAR_recev = ca[['Request ID', 'PAR Requested', 'PAR Received']].copy()
PAR_recev['PAR Requested'] = format_date_column(PAR_recev['PAR Requested'])
PAR_recev['PAR Received'] = format_date_column(PAR_recev['PAR Received'])
PAR_recev = PAR_recev[~((PAR_recev['PAR Requested'] == 'Empty date') & (PAR_recev['PAR Received'] == 'Empty date'))]
PAR_recev = PAR_recev[~((PAR_recev['PAR Requested'] == 'Empty date') | (PAR_recev['PAR Received'] == 'Empty date'))]
PAR_recev = PAR_recev[PAR_recev['PAR Requested'] > PAR_recev['PAR Received']]
PAR_recev['PAR Received'] = 'date older that PAR Requested'
PAR_recev.drop(axis=1, columns=['PAR Requested'], inplace=True)

In [ ]:
# Date Analysis

dat_nalys = ca[['Request ID', 'PAR Received', 'Date Analysis']].copy()
dat_nalys['Date Analysis'] = format_date_column(dat_nalys['Date Analysis'])
dat_nalys['PAR Received'] = format_date_column(dat_nalys['PAR Received'])
dat_nalys = dat_nalys[~((dat_nalys['PAR Received'] == 'Empty date') & (dat_nalys['Date Analysis'] == 'Empty date'))]
dat_nalys = dat_nalys[~((dat_nalys['PAR Received'] == 'Empty date') | (dat_nalys['Date Analysis'] == 'Empty date'))]
dat_nalys = dat_nalys[dat_nalys['PAR Received'] > dat_nalys['Date Analysis']]
dat_nalys['Date Analysis'] = 'date older that PAR Received'
dat_nalys.drop(axis=1, columns=['PAR Received'], inplace=True)

In [ ]:
# Pre Readout Effort

def first_character(value):
    char = 'A'
    if pd.notna(value):
        if value[0] != ' ':
            char = value[0]
        else:
            char = value[1]
    else:
        char = value

    return char

pre_reffor = ca[['Request ID', 'Pre-Readout Effort (min)', 'Account Stage']].copy()
pre_reffor['A'] = pre_reffor['Account Stage'].apply(first_character)
pre_reffor['Pre-Readout Effort (min)'] = pre_reffor['Pre-Readout Effort (min)'].apply(lambda x: x.replace(',','.') if type(x) == str else x)
pre_reffor['Pre-Readout Effort (min)'] = pre_reffor['Pre-Readout Effort (min)'].apply(lambda x: float(x) if type(x) == str else x)

def pre_effort (a,b):
    ans = a
    if pd.isna(a) and b == 'F':
        ans = 'Empty value'
    if a > 180:
        ans = 'value above 180'
    return ans

pre_reffor['Pre-Readout Effort (min)'] = pre_reffor[['Pre-Readout Effort (min)','A']].apply(lambda x: pre_effort(x[0],x[1]), axis=1)
pre_reffor = pre_reffor[(pre_reffor['Pre-Readout Effort (min)']=='Empty value') | (pre_reffor['Pre-Readout Effort (min)']=='value above 180')]
pre_reffor.drop(axis=1,columns=['Account Stage','A'],inplace=True)

In [ ]:
# Date Inite Sent

date_init_s = ca[['Request ID', 'Date First Contact', 'Date Invite Sent']].copy()
date_init_s['Date Invite Sent'] = format_date_column(date_init_s['Date Invite Sent'])
date_init_s['Date First Contact'] = format_date_column(date_init_s['Date First Contact'])
date_init_s = date_init_s[~((date_init_s['Date First Contact'] == 'Empty date') & (date_init_s['Date Invite Sent'] == 'Empty date'))]
date_init_s = date_init_s[~((date_init_s['Date First Contact'] == 'Empty date') | (date_init_s['Date Invite Sent'] == 'Empty date'))]
date_init_s = date_init_s[~(date_init_s['Date First Contact'] == 'Format Error')]
format_errors = date_init_s[date_init_s['Date Invite Sent'] == 'Format Error']
date_init_s = date_init_s[~(date_init_s['Date Invite Sent'] == 'Format Error')]
date_init_s = date_init_s[date_init_s['Date First Contact'] > date_init_s['Date Invite Sent']]
date_init_s['Date Invite Sent'] = 'date older that Date First Contact'
date_init_s = pd.concat([date_init_s, format_errors], axis=0)
date_init_s.drop(axis=1, columns=['Date First Contact'], inplace=True)

In [ ]:
# Date Schedule

date_sche = ca[['Request ID', 'Date Invite Sent', 'Date Scheduled', 'Date Readout Done']].copy()
date_sche['Date Invite Sent'] = format_date_column(date_sche['Date Invite Sent'])
date_sche['Date Scheduled'] = format_date_column(date_sche['Date Scheduled'])
date_sche['Date Readout Done'] = format_date_column(date_sche['Date Readout Done'])
date_sche = date_sche[~((date_sche['Date Invite Sent'] == 'Empty date') | (date_sche['Date Scheduled'] == 'Empty date') | (date_sche['Date Readout Done'] == 'Empty date'))]
date_sche = date_sche[~((date_sche['Date Scheduled'] >= date_sche['Date Invite Sent']) & (date_sche['Date Scheduled'] == date_sche['Date Readout Done']))]
date_sche['Date Scheduled'] = 'Relation error with Date Invite Sent or Date Readout Done'
date_sche.drop(axis=1, columns=['Date Invite Sent','Date Readout Done'], inplace=True)

In [ ]:
# Date Readout Done

def date_done(a,b,c):
    ans = b
    if type(a) == datetime and type(b) == datetime:
        if a != b:
            ans = 'different to Date Scheduled'
    elif type(a) == datetime:       
        if (a <= datetime.today()) and (b == 'Empty date'):
            ans = 'Empty value' 
    if  (c != 'F') and (b == 'Empty date'):
        ans = 'Empty value'
    return ans

date_red_don = ca[['Request ID', 'Date Scheduled', 'Date Readout Done', 'Status']].copy()
date_red_don['Date Scheduled'] = format_date_column(date_red_don['Date Scheduled'])
date_red_don['Date Readout Done'] = format_date_column(date_red_don['Date Readout Done'])
date_red_don['Status'] = date_red_don['Status'].apply(first_character)
date_red_don['Date Readout Done'] = date_red_don[['Date Scheduled','Date Readout Done','Status']].apply(lambda x: date_done(x[0],x[1],x[2]), axis=1)
date_red_don = date_red_don[~(date_red_don['Date Readout Done'] == 'Empty date')]
date_red_don = date_red_don[~(date_red_don['Date Readout Done'].apply(type) == datetime)]
date_red_don.drop(axis=1, columns=['Date Scheduled','Status'],inplace=True)

In [ ]:
# Readout Duration (min)

read_min = ca[['Request ID', 'Date Readout Done', 'Readout Duration (min)']].copy()
read_min['Readout Duration (min)'] = read_min['Readout Duration (min)'].apply(lambda x: x.replace(',','.') if type(x) == str else x)
read_min['Readout Duration (min)'] = read_min['Readout Duration (min)'].apply(float)

for j, row in read_min.iterrows():
    if pd.isnull(row['Date Readout Done']) == False and pd.isnull(row['Readout Duration (min)']) == True:
        read_min.loc[[j], ['Readout Duration (min)']] = 'Value mustnt be empty'
    elif row['Readout Duration (min)'] > 180:
        read_min.loc[[j], ['Readout Duration (min)']] = 'Value to high'
    else:
        read_min.loc[[j], ['Readout Duration (min)']] = None

read_min.dropna(axis=0, subset=['Readout Duration (min)'], inplace=True)
read_min.drop(axis=1, columns=['Date Readout Done'], inplace=True)

In [ ]:
# Account Stage // Status

c_status = ca[['Request ID', 'Account Stage', 'Status']].copy()
c_status['Charecter Stage'] = c_status['Account Stage'].apply(first_character)
c_status['Charecter Status'] = c_status['Status'].apply(first_character)
c_status = c_status[c_status['Charecter Stage'] != c_status['Charecter Status']]
c_status[['Account Stage','Status']] = c_status[['Account Stage','Status']].fillna('Empty value')  
c_status[['Account Stage','Status']] = c_status[['Account Stage','Status']].applymap(lambda x: 'different Stage vs Status' if x != 'Empty value' else x)
c_status.drop(axis=1, columns=['Charecter Stage','Charecter Status'], inplace=True)


In [ ]:
# Delivery Type

def delivery_type(col1,col2,col3,col4):
    ans = col4
    if col2 == 'B_PAR_Creation' and col3 == 'B_Unqualified - duplicated':
        if col4 != 'Not presented':
            ans = 'should be: Not presented'
    if type(col1) == datetime:
        if col1 > datetime.today():
            if col4 != 'Pending':
               ans = 'should be: Pending'
    if pd.isna(col4) == True:
        ans = 'Empty value'

    return ans

del_type = ca[['Request ID', 'Date Readout Done','Account Stage', 'Status', 'Delivery Type']].copy()
del_type['Date Readout Done'] = format_date_column(del_type['Date Readout Done'])
del_type['Delivery Type'] = del_type[['Date Readout Done','Account Stage', 'Status', 'Delivery Type']].apply(lambda i: delivery_type(i[0],i[1],i[2],i[3]), axis=1)
del_type = del_type[(del_type['Delivery Type'] == 'should be: Not presented') |
                    (del_type['Delivery Type'] == 'should be: Pending') |
                    (del_type['Delivery Type'] == 'Empty value')]
del_type = del_type[['Request ID','Delivery Type']]

In [ ]:
# Reason Declined

reason_decl = ca[['Request ID', 'Status', 'Reason Declined']].copy()

for index, row in reason_decl.iterrows():
    if pd.isnull(row['Status']) == True:
        reason_decl.loc[[index], ['Reason Declined']] = None
    elif row['Status'][2] == 'D' and pd.isnull(row['Reason Declined']) == True:
        reason_decl.loc[[index], ['Reason Declined']] = 'Empty Value'
    else:
        reason_decl.loc[[index], ['Reason Declined']] = None

reason_decl.dropna(axis=0, subset=['Reason Declined'], inplace=True)
reason_decl.drop(axis=1, columns=['Status'], inplace=True) 


In [ ]:
# Reason Declined Lvl 2

reason_decl2 = ca[['Request ID', 'Reason Declined', 'Reason Declined Lvl 2']].copy()

for index, row in reason_decl2.iterrows():
    if pd.isnull(row['Reason Declined']) == False and pd.isnull(row['Reason Declined Lvl 2']) == True:
        reason_decl2.loc[[index], ['Reason Declined Lvl 2']] = 'Empty Value'
    else:
        reason_decl2.loc[[index], ['Reason Declined Lvl 2']] = None

reason_decl2.dropna(axis=0, subset=['Reason Declined Lvl 2'], inplace=True)
reason_decl2.drop(axis=1, columns=['Reason Declined'], inplace=True) 

In [ ]:
# Reason On Hold and Days on Hold

reas_hold = ca[['Request ID', 'Reason On Hold', 'Days on Hold']].copy()

for index, row in reas_hold.iterrows():
    if pd.isna(row['Reason On Hold']) == True and pd.isna(row['Days on Hold']) == False:
        reas_hold.loc[[index], ['Reason On Hold']] = 'Empty Value'
        reas_hold.loc[[index], ['Days on Hold']] = None
    elif pd.isna(row['Reason On Hold']) == False and pd.isna(row['Days on Hold']) == True:
        reas_hold.loc[[index], ['Days on Hold']] = 'Empty Value'
        reas_hold.loc[[index], ['Reason On Hold']] = None
    else:
        reas_hold.loc[[index], ['Reason On Hold']] = None
        reas_hold.loc[[index], ['Days on Hold']] = None

reas_hold.dropna(axis=0, how='all', subset=['Reason On Hold', 'Days on Hold'], inplace=True) 

In [ ]:
# Miss Attach Opportunity // Expired Opportunity // LDoS Opportunity

oppty= ca[['Request ID', 'Account Stage','Miss Attach Opportunity', 'Expired Opportunity', 'LDoS Opportunity']].copy()

for p, row in oppty.iterrows():
    if pd.isna(row['Account Stage']) == False:
        if pd.isna(row['Miss Attach Opportunity']) == True:
            oppty.loc[[p], ['Miss Attach Opportunity']] = 'Empty oppotunity'
        if pd.isna(row['Expired Opportunity']) == True:
            oppty.loc[[p], ['Expired Opportunity']] = 'Empty oppotunity'
        if pd.isna(row['LDoS Opportunity']) == True:
            oppty.loc[[p], ['LDoS Opportunity']] = 'Empty oppotunity'

for index, row in oppty[oppty.columns[2:]].iterrows():
    for loc, sheet in row.items():
        if sheet != 'Empty oppotunity':
            oppty.loc[[index],[loc]] = None

oppty.dropna(axis=0, how='all', subset=['Miss Attach Opportunity', 'Expired Opportunity', 'LDoS Opportunity'], inplace=True)
oppty.drop(axis=1, columns=['Account Stage'], inplace=True)

In [ ]:
#next step

n_steps= ca[['Request ID', 'Account Stage', 'Status', 'Next steps']].copy()
n_steps = n_steps[(n_steps['Account Stage'] == 'B_PAR_Creation') & (n_steps['Status'] == 'B_Unqualified - duplicated')]
n_steps = n_steps[pd.isna(n_steps['Next steps'])]
n_steps['Next steps'] = 'Empty value'
n_steps = n_steps[['Request ID','Next steps']]

In [ ]:
# Follow up 1

def follo1_log(a,b):
    ans = b
    if type(a) == datetime and type(b) == datetime:
        if a > b:
            ans = 'date older that Date Readout Done'
    return ans

follow1 = ca[['Request ID', 'Date Readout Done','Date Follow Up 1']].copy()
follow1['Date Readout Done'] = format_date_column(follow1['Date Readout Done'])
follow1['Date Follow Up 1'] = format_date_column(follow1['Date Follow Up 1'])
follow1 = follow1[~(follow1['Date Readout Done'] == 'Empty date')]
follow1['Date Follow Up 1'] = follow1[['Date Readout Done','Date Follow Up 1']].apply(lambda x: follo1_log(x[0],x[1]), axis=1)
follow1 = follow1[~(follow1['Date Follow Up 1'].apply(type) == datetime)]
follow1.drop(axis=1, columns=['Date Readout Done'], inplace=True)
follow1 = follow1[~(follow1['Date Follow Up 1'] == 'Empty date')]

In [ ]:
# Follow up 2-3 validation function

def column_comparison(a,b,c):
    ans = 'N/A'
    if (type(a) == datetime) and (type(b) == datetime) and (type(c) == datetime):        
        if (a < b) and (b < c):
            ans = 'correct'    
        elif (a > b) and (b > c):
            ans = 'relation error'           
        elif a > b:
            ans = 'Previus column date is greater'            
        elif b > c:
            ans = 'Closed Date is lower'            
        else:
            ans = 'correct'

    elif (type(a) == datetime) and (type(b) == datetime):
        if a > b:
            ans = 'Previus column date is greater'
        else: 
            ans = 'correct'
    
    elif b == 'Format Error':
        ans = 'Format Error'

    return ans

In [ ]:
# Follow up 2

follow2 = ca[['Request ID', 'Date Follow Up 1', 'Date Follow Up 2','Closed Date']].copy()
follow2['Date Follow Up 1'] = format_date_column(follow2['Date Follow Up 1'])
follow2['Date Follow Up 2'] = format_date_column(follow2['Date Follow Up 2'])
follow2['Closed Date'] = format_date_column(follow2['Closed Date'])
follow2 = follow2[~(follow2['Date Follow Up 2'] == 'Empty date')]
follow2 = follow2[~(follow2['Date Follow Up 1'] == 'Empty date')]
follow2 = follow2[~(follow2['Date Follow Up 1'] == 'Format Error')]
follow2['Date Follow Up 2'] = follow2[['Date Follow Up 1','Date Follow Up 2','Closed Date']].apply(lambda row : column_comparison(row[0],row[1],row[2]), axis=1) 
follow2 = follow2[~(follow2['Date Follow Up 2'] == 'correct')]
follow2.drop(axis=1, columns=['Date Follow Up 1', 'Closed Date'], inplace=True)

In [ ]:
# Follow up 3

follow3 = ca[['Request ID', 'Date Follow Up 2', 'Date Follow Up 3','Closed Date']].copy()
follow3['Date Follow Up 2'] = format_date_column(follow3['Date Follow Up 2'])
follow3['Date Follow Up 3'] = format_date_column(follow3['Date Follow Up 3'])
follow3['Closed Date'] = format_date_column(follow3['Closed Date'])
follow3 = follow3[~(follow3['Date Follow Up 3'] == 'Empty date')]
follow3 = follow3[~(follow3['Date Follow Up 2'] == 'Empty date')]
follow3 = follow3[~(follow3['Date Follow Up 2'] == 'Format Error')]
follow3['Date Follow Up 3'] = follow3[['Date Follow Up 2','Date Follow Up 3','Closed Date']].apply(lambda row : column_comparison(row[0],row[1],row[2]), axis=1) 
follow3 = follow3[~(follow3['Date Follow Up 3'] == 'correct')]
follow3.drop(axis=1, columns=['Date Follow Up 2', 'Closed Date'], inplace=True)

In [ ]:
# Follow Up Effort (min)

follow_eft = ca[['Request ID', 'Date Readout Done','Date Follow Up 1','Follow Up Effort (min)']].copy()
follow_eft = follow_eft[~(follow_eft['Follow Up Effort (min)'] == 0)]
follow_eft['Follow Up Effort (min)'] = follow_eft['Follow Up Effort (min)'].fillna(0)
follow_eft['Follow Up Effort (min)'] = follow_eft['Follow Up Effort (min)'].replace(' ',0)
follow_eft['Follow Up Effort (min)'] = follow_eft['Follow Up Effort (min)'].apply(lambda x: x.replace(',','.') if type(x) == str else x)
follow_eft['Follow Up Effort (min)'] = follow_eft['Follow Up Effort (min)'].apply(lambda x: float(x) if type(x) == str else x)
follow_eft = follow_eft[(follow_eft['Follow Up Effort (min)'] == 0) | (follow_eft['Follow Up Effort (min)'] > 240)]
follow_eft = follow_eft[(pd.notna(follow_eft['Date Readout Done']) == True) | (pd.notna(follow_eft['Date Follow Up 1']) == True) | (follow_eft['Follow Up Effort (min)'] != 0)]
follow_eft['Follow Up Effort (min)'] = follow_eft['Follow Up Effort (min)'].replace(0, 'Empty value')
follow_eft['Follow Up Effort (min)'] = follow_eft['Follow Up Effort (min)'].apply(lambda x: 'Value too high' if type(x) == float else x)
follow_eft = follow_eft[['Request ID','Follow Up Effort (min)']]

In [ ]:
# Closing Date
def close_date(colm1,colm3):
    ans = colm3
    if type(colm1) == Timestamp and type(colm3) == datetime: 
        if colm1 > colm3:
            ans = 'date is older than Date Received'
    if colm3 == 'Empty date':
        ans = 'Empty value'
    return ans


closig_dat = ca[['Request ID', 'Date received', 'Reason Declined','Reason Declined Lvl 2','Account Stage', 'Status', 'Date Follow Up 3', 'Closed Date']].copy()
closig_dat = closig_dat[~(((pd.notna(closig_dat['Reason Declined'])) | (pd.notna(closig_dat['Reason Declined Lvl 2']))) & (pd.isna(closig_dat['Closed Date'])))]
closig_dat = closig_dat[(closig_dat['Account Stage'] == 'B_PAR_Creation') & (closig_dat['Status'] == 'B_Unqualified-duplicated')]
closig_dat = closig_dat[~((pd.notna(closig_dat['Reason Declined'])) | (pd.notna(closig_dat['Reason Declined Lvl 2'])))]
closig_dat['Date received'] = format_date_column(closig_dat['Date received'])
closig_dat['Closed Date'] = format_date_column(closig_dat['Closed Date'])
closig_dat['Closed Date'] = closig_dat[['Date received', 'Closed Date']].apply(lambda row: close_date(row[0],row[1]), axis=1)
closig_dat = closig_dat[~(closig_dat['Closed Date'] == 'Empty date')]
closig_dat = closig_dat[closig_dat['Closed Date'].apply(type) == str]
closig_dat.drop(axis=1, columns=['Date received','Reason Declined','Reason Declined Lvl 2','Account Stage','Status', 'Date Follow Up 3'], inplace=True)


### Merging tables

In [ ]:
# Merging new tables
new_table = request_type.merge(join_call, how='outer', left_on='Request ID', right_on='Request ID').merge(
            upcoming_renewal, how='outer', left_on='Request ID', right_on='Request ID').merge(
            date_received, how='outer', left_on='Request ID', right_on='Request ID').merge(
            cov_adv, how='outer', left_on='Request ID', right_on='Request ID').merge(
            PAR_req, how='outer', left_on='Request ID', right_on='Request ID').merge(
            d_f_contact, how='outer', left_on='Request ID', right_on='Request ID').merge(
            PAR_recev, how='outer', left_on='Request ID', right_on='Request ID').merge(
            dat_nalys, how='outer', left_on='Request ID', right_on='Request ID').merge(
            pre_reffor, how='outer', left_on='Request ID', right_on='Request ID').merge(
            date_init_s, how='outer', left_on='Request ID', right_on='Request ID').merge(
            date_sche, how='outer', left_on='Request ID', right_on='Request ID').merge(
            date_red_don, how='outer', left_on='Request ID', right_on='Request ID').merge(
            read_min, how='outer', left_on='Request ID', right_on='Request ID').merge(
            c_status, how='outer', left_on='Request ID', right_on='Request ID').merge(
            del_type, how='outer', left_on='Request ID', right_on='Request ID').merge(
            reason_decl, how='outer', left_on='Request ID', right_on='Request ID').merge(
            reason_decl2, how='outer', left_on='Request ID', right_on='Request ID').merge(
            reas_hold, how='outer', left_on='Request ID', right_on='Request ID').merge(
            oppty, how='outer', left_on='Request ID', right_on='Request ID').merge(
            n_steps, how='outer', left_on='Request ID', right_on='Request ID').merge(   
            follow1, how='outer', left_on='Request ID', right_on='Request ID').merge(
            follow2, how='outer', left_on='Request ID', right_on='Request ID').merge(
            follow3, how='outer', left_on='Request ID', right_on='Request ID').merge(
            follow_eft, how='outer', left_on='Request ID', right_on='Request ID').merge(
            closig_dat, how='outer', left_on='Request ID', right_on='Request ID')

In [ ]:
new_table.dropna(axis=0, how='all', subset=new_table.columns[2:], inplace=True)  
t_names = pd.DataFrame()
t_names[['Request ID','Name','Account Name']] = ca[['Request ID','Coverage Advisor','Account Name']]
new_table = t_names.merge(new_table, how='right', left_on='Request ID', right_on='Request ID')

In [ ]:
mail_table = new_table['Name'].to_frame().drop_duplicates().reset_index(drop=True)
mail_table = mail_table.merge(mail, how='left', left_on='Name', right_on='Coverage Advisor')
mail_table.dropna(axis=0, subset=['Email'], inplace=True)

### Send Mail

In [ ]:
#outlook server connection
outlServer = smtplib.SMTP(host='smtp-mail.outlook.com', port=587)

#Initialize encryption
outlServer.starttls()

#Login in the outlook account using the define variables username and password
outlServer.login(myMail, password)

In [ ]:
#Definition of the function to send emails
def send_msg(To, From, Subject, Message, s_attach):
    msg = MIMEMultipart()
    msg['From'] = From
    msg['To'] = To
    msg['Subject'] = Subject
    msg_content = MIMEText(Message, 'html', 'utf-8')
    msg.attach(msg_content)
       
    if s_attach == True:
        with open('error_report_%s.xlsx' % today, 'rb') as e:
            exc = MIMEBase('application', 'octet-stream', filename='error_report_%s.xlsx' % today)
            exc.add_header('content-Disposition', 'attachment', filename='error_report_%s.xlsx' % today)
            exc.set_payload(e.read())
            encoders.encode_base64(exc)
            msg.attach(exc)
    
    outlServer.sendmail(From, To, msg.as_string())

In [ ]:
for i, sheet in mail_table.iterrows():
    if pd.isnull(sheet['Email']) == False:
        df_sent = new_table[new_table['Name'] == sheet['Name']]
        df_sent.to_excel('error_report_%s.xlsx' % today, index=False)    
        excel_formating.format_excel()
        if df_sent.shape[0] > 0:
            send_msg(sheet['Email'], myMail, subjectEmail , msg1 % sheet['Name'], True)
            print(sheet['Email'])
        else:
            print('Not send to: '+sheet['Email'])

In [ ]:
## General report

new_table.to_excel('error_report_%s.xlsx' % today, index=False)    
excel_formating.format_excel()

In [ ]:
send_msg('jonnathan.neira@nub78.com', myMail, subjectEmail , msg1 % 'Jonnathan', True)
send_msg('maria.rojas@nub78.com', myMail, subjectEmail , msg1 % 'Maria', True)

In [ ]:
outlServer.quit()

In [ ]:
with open('nub78.txt', 'r') as i:
    print(i.read()) 